In [6]:
%%javascript

window.load_remote_theme = true
var theme_js = "https://odhk.github.io/hyrule_theme/custom.js";

window.load_local_theme = function(){
    var hostname = document.location.hostname
    return ((hostname == "localhost" || hostname == '127.0.0.1') && !load_remote_theme)
}

var url = load_local_theme() ? document.location.origin + "/files/theme/custom.js" : theme_js

$.getScript(url)

<IPython.core.display.Javascript object>

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding,constraints,\
Dropout, Activation,GRU,Bidirectional,Subtract, Permute, TimeDistributed, Reshape
from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D,Layer, initializers, regularizers
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
import gc
import tensorflow as tf
from keras.activations import softmax

from keras.utils import plot_model
from keras.layers import *
from keras.applications.imagenet_utils import _obtain_input_shape
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.special import erfinv

import gensim
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

label = train['class']

char_embedding_index = {}
f = open('word_embedding_300dim_new2.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0].lower()
    embeddings = np.asarray(values[1:], dtype = 'float32')
    char_embedding_index[word] = embeddings
f.close()

len(char_embedding_index)

api_embedding_file_path = 'word_embedding_300dim_new2.txt'
embedding_dim = 300
max_nb_api = len(char_embedding_index) + 1
max_sequence_length_api =1200

all_word = list(train['word_seg'].values) + list(test['word_seg'].values)

all_word_list = []
for i in list(all_word):
    all_word_list.append(i.split(' '))

len(all_word_list)

train['word_seg'].apply(lambda x:len(x.split(' '))).describe()

data = pd.DataFrame()

gc.collect()

tokenizer_api = Tokenizer()

tokenizer_api.fit_on_texts(all_word_list)

api_seq = tokenizer_api.texts_to_sequences(all_word_list)

data['word_seq'] = pad_sequences(api_seq, maxlen = max_sequence_length_api, padding='post').tolist()

api_index = tokenizer_api.word_index
print('Found %s unique tokens' % len(api_index))
nb_words = min(max_nb_api, len(char_embedding_index)+1)
api_embedding_matrix = np.zeros((nb_words, embedding_dim))

for word, i in tqdm(api_index.items()):
    embedding_vector = char_embedding_index.get(word)
    if embedding_vector is not None:
        api_embedding_matrix[i] = embedding_vector
    else:
        print('no char%s'%word)

data['id'] = train['id']

train_FE = data.iloc[:len(label),:]
test_FE = data.iloc[len(label):,:]

os.environ["CUDA_VISIBLE_DEVICES"] = "7"

def get_input(df):
    
    _input = [np.array(df.word_seq.values.tolist())]
    
    
    return _input 

/home/libo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
1271461it [01:45, 12058.38it/s]
  2%|▏         | 30100/1271460 [00:00<00:04, 300937.87it/s]

Found 1271460 unique tokens


100%|██████████| 1271460/1271460 [00:04<00:00, 287511.47it/s]


In [23]:
def RNN1():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    
    #RNN
    char_rnn_layer_1 = Bidirectional(CuDNNLSTM(250, 
                               return_sequences=True))
    char_rnn_layer_2 = Bidirectional(CuDNNLSTM(250, 
                               return_sequences=True))
    
    char_embedding = api_embedding_layer(char_input)
    #char_embedding = SpatialDropout1D(0.2)(char_embedding)
    
    #双层BiLSTM
    char_rnn_1= char_rnn_layer_1(char_embedding)
    char_rnn_2= char_rnn_layer_2(char_rnn_1)
    
    #max avg min
    char_att_maxpooling_1 = GlobalMaxPooling1D()(char_rnn_1)
    char_att_avgpooling_1 = GlobalAveragePooling1D()(char_rnn_1)
    
    char_att_maxpooling_2 = GlobalMaxPooling1D()(char_rnn_2)
    char_att_avgpooling_2 = GlobalAveragePooling1D()(char_rnn_2)
    
    sub_char_abs_1 = Lambda(lambda x:K.abs(x[0] - x[1]))([char_att_maxpooling_1, char_att_avgpooling_1])
    sub_char_abs_2 = Lambda(lambda x:K.abs(x[0] - x[1]))([char_att_maxpooling_2, char_att_avgpooling_2])
    
    merged = concatenate([char_att_maxpooling_1, char_att_avgpooling_1, \
                          char_att_maxpooling_2, char_att_avgpooling_2, sub_char_abs_1, sub_char_abs_2])
    
    merged = Dropout(0.2)(merged)
    merged = BatchNormalization()(merged)
    merged = Dense(400, activation='relu')(merged)
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged) 
    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [24]:
meta_train = pd.DataFrame(np.array([None]*train_FE.shape[0]*19).reshape(train_FE.shape[0], 19))
meta_train.columns = ['prob' + str(x) for x in range(19)]
meta_test = pd.DataFrame()

label = train['class']

label = label - 1

categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=19)

index = pd.read_csv('fold.csv')
index['id'] = train['id']

In [ ]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
#     bst_model_path = '10folds_' + 'base_rnn_wrod_v1_300dim' + str(i) + '.hdf5'
#     early_stopping =EarlyStopping(monitor='val_acc', patience=2)
#     model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
#     callbacks = [
#             early_stopping,
#             model_checkpoint
#         ]
    model = RNN1()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=10, batch_size=BATCH_SIZE, shuffle=True)#callbacks=callbacks
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/20
92040/92040 [==============================] - 468s 5ms/step - loss: 1.0156 - acc: 0.7062 - val_loss: 0.9709 - val_acc: 0.7060
Epoch 2/20
92040/92040 [==============================] - 403s 4ms/step - loss: 0.8158 - acc: 0.7566 - val_loss: 0.8605 - val_acc: 0.7512
Epoch 3/20
92040/92040 [==============================] - 403s 4ms/step - loss: 0.7471 - acc: 0.7751 - val_loss: 0.7859 - val_acc: 0.7680
Epoch 4/20
92040/92040 [==============================] - 403s 4ms/step - loss: 0.6888 - acc: 0.7899 - val_loss: 0.8150 - val_acc: 0.7598
Epoch 5/20
102277/102277 [==============================] - 129s 1ms/step
*******1*******
Train on 92043 samples, validate on 10234 samples
Epoch 1/20
92043/92043 [==============================] - 412s 4ms/step - loss: 1.0126 - acc: 0.7041 - val_loss: 1.0021 - val_acc: 0.7101
Epoch 2/20
92043/92043 [==============================] - 427s 5ms/step - loss: 0.8098 - acc: 0.7586 - v

In [11]:
meta_train

,meta_base_rnn_wrod_v1_300dim_prob0,meta_base_rnn_wrod_v1_300dim_prob1,meta_base_rnn_wrod_v1_300dim_prob2,meta_base_rnn_wrod_v1_300dim_prob3,meta_base_rnn_wrod_v1_300dim_prob4,meta_base_rnn_wrod_v1_300dim_prob5,meta_base_rnn_wrod_v1_300dim_prob6,meta_base_rnn_wrod_v1_300dim_prob7,meta_base_rnn_wrod_v1_300dim_prob8,meta_base_rnn_wrod_v1_300dim_prob9,meta_base_rnn_wrod_v1_300dim_prob10,meta_base_rnn_wrod_v1_300dim_prob11,meta_base_rnn_wrod_v1_300dim_prob12,meta_base_rnn_wrod_v1_300dim_prob13,meta_base_rnn_wrod_v1_300dim_prob14,meta_base_rnn_wrod_v1_300dim_prob15,meta_base_rnn_wrod_v1_300dim_prob16,meta_base_rnn_wrod_v1_300dim_prob17,meta_base_rnn_wrod_v1_300dim_prob18
0,0.843309,0.00385805,0.000102125,0.000280596,0.00113676,0.000297191,0.00201609,0.011277,0.000228425,0.0279983,0.000527547,0.000457277,0.000981998,0.0844372,0.00138605,0.00620003,0.00540942,0.00878303,0.00131441
1,0.0151885,0.00205575,0.205176,0.0150657,0.00104939,0.0114522,0.0107847,0.0178365,0.00477458,0.0044505,0.0104938,0.00337448,0.0326517,0.000376994,0.561974,0.00221263,0.00721158,0.057818,0.0360538
2,0.00957131,0.000227768,0.000799553,0.0167076,0.000784,0.000451077,0.263447,0.000484068,0.00258696,0.00594361,0.00607816,0.00684792,0.00824489,0.673102,0.00141714,3.83659e-05,0.000402002,0.000100433,0.00276673
3,0.0123792,0.00113402,5.87818e-05,0.00197263,0.000478276,6.4085e-05,0.000301501,0.000593545,0.00014242,0.000111584,5.6173e-05,0.00204868,0.680166,0.280089,9.91731e-05,0.015348,0.0021364,0.000472363,0.00234889
4,0.00254471,6.4346e-05,0.000528202,0.318447,0.000129541,0.000277433,0.0834307,0.000307498,0.0101595,5.16035e-05,0.00849455,0.55004,0.00080656,0.0181468,0.00503975,2.06945e-05,0.00135884,1.57882e-05,0.000136033
5,0.0184902,0.000422167,0.0100747,0.000687973,0.000124711,7.7079e-05,0.000932308,0.000122889,8.12786e-05,0.000491437,0.0296258,0.00278336,0.922409,0.000342142,0.00279051,0.000950382,3.25806e-05,0.000489108,0.00907236
6,0.788332,0.000334602,0.0185646,0.000574465,0.004053,0.000226971,0.0038321,0.0124038,0.00227509,0.0856138,0.0059898,0.0026417,0.000129325,0.0362188,0.00121623,3.37266e-05,0.0239614,0.00978395,0.00381497
7,7.48919e-05,2.31249e-05,0.000202785,1.63342e-07,8.26145e-06,0.10912,8.14359e-06,0.000471771,5.30261e-06,0.889522,2.40785e-05,0.000184653,0.00018651,1.16147e-06,4.15998e-07,8.72374e-05,1.8724e-05,2.746e-06,5.79916e-05
8,0.0014666,0.000106759,3.04273e-05,2.22436e-05,0.000194526,3.09678e-05,8.60485e-05,0.0013385,2.27349e-05,0.996469,1.7496e-05,2.93614e-05,4.91412e-05,9.46555e-06,2.37934e-06,2.15814e-05,1.71409e-05,7.73404e-06,7.78797e-05
9,0.0673959,0.000113092,0.00371484,3.35084e-05,0.000685924,7.06593e-05,0.184815,0.000716093,0.0247319,0.000611504,0.0190444,0.156417,0.0320471,0.0458184,0.000870905,9.7643e-06,0.000411843,0.00335388,0.459139


In [ ]:
meta_train.columns = ['meta_base_rnn_wrod_v1_300dim_' + x for x in meta_train.columns]

p = pd.DataFrame()

for i in range(19):
    p[i] = meta_test[i].mean(axis = 1)

p.columns = meta_train.columns

In [17]:
meta_train.to_csv('/home/libo/daguan/stack_new/train_meta_base_rnn_word_v1_300dim.csv', index = None ,encoding = 'utf-8')

p.to_csv('/home/libo/daguan/stack_new/test_meta_base_rnn_word_v1_300dim.csv', index = None ,encoding = 'utf-8')

In [11]:
def RNN():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    char_embedding = api_embedding_layer(char_input)
#     with tf.device('/cpu:0'):
#         char_embedding = api_embedding_layer(char_input)

    
    char_rnn_1= Bidirectional(CuDNNLSTM(100, 
                                return_sequences=True))(char_embedding)
    char_rnn_2= Bidirectional(CuDNNLSTM(100, 
                                return_sequences=True))(char_rnn_1)
    
#     char_rnn_1 = Lambda(lambda x:K.permute_dimensions(x, (0, 2, 1)))(char_rnn_1)
#     char_rnn_2 = Lambda(lambda x:K.permute_dimensions(x, (0, 2, 1)))(char_rnn_2)

    merged = concatenate([char_rnn_1, char_rnn_2], axis = -1)
    
    #merged = Lambda(lambda x:K.permute_dimensions(x, (0, 2, 1)))(merged)
    
    merged = Dense(1024, activation='relu')(merged)

    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [12]:
model = RNN()

In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 1200)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1200, 300)    381438300   input_4[0][0]                    
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 1200, 200)    321600      embedding_4[0][0]                
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 1200, 200)    241600      bidirectional_5[0][0]            
__________________________________________________________________________________________________
concatenat

In [14]:
meta_train = pd.DataFrame(np.array([None]*train_FE.shape[0]*19).reshape(train_FE.shape[0], 19))
meta_train.columns = ['prob' + str(x) for x in range(19)]
meta_test = pd.DataFrame()

label = train['class']

label = label - 1

categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=19)

index = pd.read_csv('fold.csv')
index['id'] = train['id']

In [20]:
def RNN():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    char_embedding = api_embedding_layer(char_input)
#     with tf.device('/cpu:0'):
#         char_embedding = api_embedding_layer(char_input)

    
    char_rnn_1= Bidirectional(CuDNNLSTM(100, 
                                return_sequences=True))(char_embedding)
    char_rnn_2= Bidirectional(CuDNNLSTM(100, 
                                return_sequences=True))(char_rnn_1)
    
#     char_rnn_1 = Lambda(lambda x:K.permute_dimensions(x, (0, 2, 1)))(char_rnn_1)
#     char_rnn_2 = Lambda(lambda x:K.permute_dimensions(x, (0, 2, 1)))(char_rnn_2)

    #merged = concatenate([char_rnn_1, char_rnn_2], axis = -1)
    
    #merged = Lambda(lambda x:K.permute_dimensions(x, (0, 2, 1)))(merged)
    
    merged = Dense(1024, activation='relu')(char_rnn_1)

    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [33]:
def RNN():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    
    #RNN
    char_rnn_layer_1 = Bidirectional(CuDNNLSTM(250, 
                               return_sequences=True))
    char_rnn_layer_2 = Bidirectional(CuDNNLSTM(250, 
                               return_sequences=True))
    
    char_embedding = api_embedding_layer(char_input)
#     with tf.device('/cpu:0'):
#         char_embedding = api_embedding_layer(char_input)
    #char_embedding = SpatialDropout1D(0.2)(char_embedding)
    
    #双层BiLSTM
    char_rnn_1= char_rnn_layer_1(char_embedding)
    char_rnn_2= char_rnn_layer_2(char_rnn_1)
    char_rnn_2 = Flatten()(char_rnn_2)
    merged = Dropout(0.2)(char_rnn_2)
    merged = BatchNormalization()(merged)
    merged = Dense(400, activation='relu')(merged)
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged) 
    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [35]:
model = RNN()

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1200)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1200, 300)         381438300 
_________________________________________________________________
bidirectional_3 (Bidirection (None, 1200, 500)         1104000   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 1200, 500)         1504000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 600000)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 600000)            0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 600000)            2400000   
__________

In [37]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
    bst_model_path = '10folds_' + 'base_rnn_wrod_v1_300dim' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_acc', patience=2)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = RNN()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=20, batch_size=BATCH_SIZE, shuffle=True, callbacks=callbacks)#callbacks=callbacks
    
    model = RNN()
    model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=64,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=64,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor with shape[600000,400] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/Adam/gradients/dense_1/MatMul_grad/MatMul_1 = MatMul[T=DT_FLOAT, _class=["loc:@dense_1/MatMul"], transpose_a=true, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](batch_normalization_1/cond/Merge, training/Adam/gradients/dense_1/Relu_grad/ReluGrad)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training/Adam/gradients/dense_1/MatMul_grad/MatMul_1', defined at:
  File "/home/libo/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/libo/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/libo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, 

In [19]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1200)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1200, 300)    381438300   input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1200, 200)    321600      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 1200, 200)    241600      bidirectional_1[0][0]            
__________________________________________________________________________________________________
concatenat